# Predictions on "Risk Factors of cervical cancer" dataset
Authors: Kevin Lagos, Sonny Muñoz, María Retamal

#### Lectura de los paquetes que se utilizarán

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import io 
from google.colab import files
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
%matplotlib inline

#para métricas
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
metricas = []

### Carga y Manipulación de datos

In [ ]:
# Fijamos la semilla que genera numeros aleatorios
np.random.seed(0)

#Carga de los datos
uploaded = files.upload()
df = pd.read_csv(io.StringIO(uploaded['kag_risk_factors_cervical_cancer.csv'].decode('utf-8')))

#Cambio de "?" por NaN y paso a tipo float
for col in df.columns: 
    df.loc[df[col]=='?', col]=np.nan
    if df.dtypes[col] == object:
      df = df.astype({col: float})

#Cambio en las variables target para dejarlo como 1 variable target binaria
df["target"] = df[["Biopsy","Hinselmann","Citology","Schiller"]].max(axis=1)
df["target"] = df["target"].astype(np.int64)
del df["Biopsy"]
del df["Hinselmann"] 
del df["Citology"] 
del df["Schiller"]

X = df.dropna()
df = df.dropna() # Borramos filas que son NaN

Saving kag_risk_factors_cervical_cancer.csv to kag_risk_factors_cervical_cancer.csv


### **Selección de características**

Para abordar el método de Selección de características se desarrollará un modelo basado en RFE (Recursive Feature Elimination), donde se eliminan características de manera recursiva y se construye un modelo con las que quedan. Se utiliza el accuraccy del modelo para identificar que atributos o combinaciones de atributos contribuyen más a predecir el target.


Separación de features y target

In [ ]:
Y = df.target #target
X = df.iloc[:, :-1] #features

División entre datos de entrenamiento y testeo

In [ ]:
seed = 1
test_size = 0.3
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

Creación del modelo

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# crea una base clasificadora usada para evaluar un subconjunto de atributos, con 250 iteraciones como máximo para converger
model = LogisticRegression(max_iter=250)

#crea el modelo RFE y selecciona 10 atributos
rfe = RFE(model, 10)
rfe = rfe.fit(X, y)


Predicciones y evaluación con datos de testeo

In [ ]:
y_pred = rfe.predict(X_test)

#evaluar predicciones
print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
print('Recall: %.2f' % recall_score(y_test, y_pred))
print('Precisión: %.2f' % precision_score(y_test, y_pred))
print('F1: %.2f' % f1_score(y_test, y_pred))
metricas.append((accuracy_score(y_test, y_pred),recall_score(y_test, y_pred),precision_score(y_test, y_pred),f1_score(y_test, y_pred)))

Accuracy: 0.83
Recall: 0.25
Precisión: 1.00
F1: 0.40


Features que fueron seleccionados

In [ ]:
selected = list()
rank = rfe.ranking_
for i in range(len(X.columns)):
  if (rank[i] == 1):
    selected.append(X.columns[i])

print("Features seleccionados:")
print(selected)

Features seleccionados:
['Smokes', 'Smokes (packs/year)', 'IUD', 'STDs (number)', 'STDs:vaginal condylomatosis', 'STDs:vulvo-perineal condylomatosis', 'STDs:syphilis', 'STDs:genital herpes', 'STDs:HIV', 'STDs: Number of diagnosis']


### **XGBoost**

XGBoost (Extreme Gradient Boosting) corresponde a un algoritmo predictivo supervisado que utiliza el principio de "boosting". La idea es generar múltiples modelos de predicción "débiles" secuencialmente, y que cada uno de estos tome los resultados del modelo anterior para volverse "fuerte", por ende con mejor poder predictivo y mayor estabilidad en resultados.

Separación de features y target

In [ ]:
Y = df.target #target
X = df.iloc[:, :-1] #features

División entre datos de entrenamiento y testeo

In [ ]:
seed = 1
test_size = 0.3
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

Creación del modelo

In [ ]:
from xgboost import XGBClassifier

# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

Predicciones y evaluación con datos de testeo

In [ ]:
y_pred = model.predict(X_test)

#evaluar predicciones
print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
print('Recall: %.2f' % recall_score(y_test, y_pred))
print('Precisión: %.2f' % precision_score(y_test, y_pred))
print('F1: %.2f' % f1_score(y_test, y_pred))
metricas.append((accuracy_score(y_test, y_pred),recall_score(y_test, y_pred),precision_score(y_test, y_pred),f1_score(y_test, y_pred)))

Accuracy: 0.50
Recall: 0.25
Precisión: 0.14
F1: 0.18



### **Deep Learning**

Se implementará una red neuronal profunda que contiene capas de nodos conectadas entre sí, para ello se hará uso de Keras.

In [ ]:
#para métricas

from keras import backend as K
import tensorflow as tf 

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


Separación de features y target

In [ ]:
Y = df.target #target
X = df.iloc[:, :-1] #features

División entre datos de entrenamiento y testeo

In [ ]:
seed = 1
test_size = 0.3
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

Definición del modelo

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(512, input_dim=32, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

Compilación del modelo

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy', tf.keras.metrics.Recall(),tf.keras.metrics.Precision(),f1_m])
model.fit(X_train, y_train, epochs=150, batch_size=10)

Epoch 1/150
5/5 [==============================] - 1s 4ms/step - loss: 0.7016 - accuracy: 0.6655 - recall_10: 0.0000e+00 - precision_10: 0.0000e+00 - f1_m: 0.0000e+00
Epoch 2/150
5/5 [==============================] - 0s 3ms/step - loss: 0.7052 - accuracy: 0.7286 - recall_10: 0.0000e+00 - precision_10: 0.0000e+00 - f1_m: 0.0000e+00
Epoch 3/150
5/5 [==============================] - 0s 4ms/step - loss: 0.5903 - accuracy: 0.7270 - recall_10: 0.1975 - precision_10: 0.4944 - f1_m: 0.2367        
Epoch 4/150
5/5 [==============================] - 0s 4ms/step - loss: 0.5676 - accuracy: 0.7189 - recall_10: 0.0000e+00 - precision_10: 0.0000e+00 - f1_m: 0.0000e+00
Epoch 5/150
5/5 [==============================] - 0s 4ms/step - loss: 0.6308 - accuracy: 0.6491 - recall_10: 0.2879 - precision_10: 0.3324 - f1_m: 0.2631        
Epoch 6/150
5/5 [==============================] - 0s 4ms/step - loss: 0.5963 - accuracy: 0.7770 - recall_10: 0.5537 - precision_10: 0.6042 - f1_m: 0.5227
Epoch 7/150
5/5 [=

Evaluación del modelo

In [ ]:
_, accuracy,recall,precision,f1_score = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy))
print('Recall: %.2f' % (recall))
print('Precisión: %.2f' % (precision))
print('F1: %.2f' % (f1_score))
metricas.append((accuracy,recall,precision,f1_score))

1/1 [==============================] - 0s 15ms/step - loss: 4.7606 - accuracy: 0.7222 - recall_10: 0.2500 - precision_10: 0.3333 - f1_m: 0.2857
Accuracy: 0.72
Recall: 0.25
Precisión: 0.33
F1: 0.29


### **Otra técnica: AdaBoost**

Otro algoritmo basado en el boosting, por lo que también se basa en el principio de generar modelos secuencialmente, con el propósito de que vayan mejorar a través de iteraciones.

Separación de features y target

In [ ]:
Y = df.target #target
X = df.iloc[:, :-1] #features

División entre datos de entrenamiento y testeo

In [ ]:
seed = 1
test_size = 0.3
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

Creación del modelo

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

model = AdaBoostClassifier(n_estimators=250, random_state=0)
model.fit(X_train, y_train)


AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=250, random_state=0)

Predicciones y evaluación con datos de testeo

In [ ]:
y_pred = model.predict(X_test)

prec =precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = (2*(prec*rec)/(prec+rec)).round(2)

#evaluar predicciones
print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
print('Recall: %.2f' % rec)
print('Precisión: %.2f' % prec)
print('F1: %.2f' % f1)
metricas.append((accuracy_score(y_test, y_pred),rec,prec,f1))

Accuracy: 0.61
Recall: 0.25
Precisión: 0.20
F1: 0.22


### **Tabla comparativa**

En la siguiente tabla se muestran las métricas de cada método 

In [ ]:
import plotly.graph_objects as go
metodos = ["RFE","XGBoost","Deep Learning","Adaboost"]
accuracy = []
recall = []
precision = []
f1 = []
tasa_e = []
for i in np.arange(4):
  accuracy.append(metricas[i][0])
  tasa_e.append(1-metricas[i][0])
  recall.append(metricas[i][1])
  precision.append(metricas[i][2])
  f1.append(metricas[i][3])

fig = go.Figure(data=[go.Table(header=dict(values=["Métodos","Accuracy","Tasa de error","Recall (Sensibilidad)","Precisión","F1"]),
                 cells=dict(values=[metodos,accuracy,tasa_e,recall,precision,f1,]))
                     ])
fig.show()

Podemos saber cuáles métodos poseen un mejor desempeño en base a sus indicadores:


*   Los métodos con mejor *Accuracy* son *Selección de características* y *Deep Learning* , además de muy cerca lo siguen *Adaboost* (se puede realizar el mismo análisis con la Tasa de error pero buscando las cifras menores).
*   Con respecto a la sensibilidad, todos los métodos presentes en este taller poseen la misma cifra *0.25*.
*   En cuanto a la precisión, se nota que el mejor desempeño lo posee *RFE (Selección de características)*, siguiendo por mucho más abajo *Deep Learning*.
*   Con respecto al indicador F1 nuevamente *Selección de características* presenta un mejor desempeño.

Por lo que, se puede concluir en base a la tabla que el método *Selección de características (RFE)* presenta un mejor desempeño con respecto a los otros métodos, debido a que muestra mejores resultados en sus indicadores, no solo en Accuracy y Precisión, además se iguala con los demás métodos en Sensibilidad, lo cual es importante para detectar casos positivos (lo cual es bastante relevante en el contexto de salud). Notar también que, si se compara el método basado en *RFE* con los vistos en el Taller 2, este se muestra como superior con gran ventaja.